In [1]:
# import libraries
from bs4 import BeautifulSoup
#import numpy as np
import requests

In [2]:
import pandas as pd 

In [2]:
url_base = 'https://codigo-postal.co/argentina/'
response = requests.get(url_base)
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [4]:
response.status_code

200

In [18]:
urlProvincias = []

In [3]:
# Encuentra los elementos que contienen los enlaces de las provincias
enlaces_provincias = soup.find_all('ul',{'class': 'column-list'})

In [33]:
enlaces_provincias

[<ul class="column-list">
 <li><a href="https://codigo-postal.co/argentina/buenos-aires/" title="Código Postal Buenos Aires"><span class="ar ar-buenos-aires"></span>Buenos Aires</a></li>
 <li><a href="https://codigo-postal.co/argentina/capital-federal/" title="Código Postal Capital Federal"><span class="ar ar-buenos-aires-capital-federal"></span>Buenos Aires (CABA)</a></li>
 <li><a href="https://codigo-postal.co/argentina/catamarca/" title="Código Postal Catamarca"><span class="ar ar-catamarca"></span>Catamarca</a></li>
 <li><a href="https://codigo-postal.co/argentina/chaco/" title="Código Postal Chacho"><span class="ar ar-chaco"></span>Chaco</a></li>
 <li><a href="https://codigo-postal.co/argentina/chubut/" title="Código Postal Chubut"><span class="ar ar-chubut"></span>Chubut</a></li>
 <li><a href="https://codigo-postal.co/argentina/cordoba/" title="Código Postal Cordoba"><span class="ar ar-cordoba"></span>Córdoba</a></li>
 <li><a href="https://codigo-postal.co/argentina/corrientes/" 

In [19]:
for elemento in enlaces_provincias[0].findAll('a'):
    urlProvincias.append(elemento['href'])

In [21]:
len(urlProvincias)

24

citis

In [25]:
url_base1 = 'https://codigo-postal.co/argentina/buenos-aires/'
response1 = requests.get(url_base1)
html1 = response1.content
soup1 = BeautifulSoup(html1, 'html.parser')

In [42]:
# Encuentra los elementos que contienen los enlaces de las provincias
enlaces_citis = soup1.find_all('ul',{'class': "cities"})

In [49]:
citiesEnlaces = []

In [50]:
for i in enlaces_citis:
    c =i.findAll('a')
    for e in c:
        citiesEnlaces.append(e['href'])

In [3]:
def cpa(provincia,localidad,calle):
    provincia = provincia.lower().replace(" ", "-")
    localidad = localidad.lower().replace(" ", "-")
    calle = calle.lower().replace(" ", "-")
    url_base1 = f'https://codigo-postal.co/argentina/{provincia}/{localidad}'
    #response1 = requests.get(url_base1)
    #html1 = response1.content
    #soup1 = BeautifulSoup(html1, 'html.parser')
    df = pd.read_html(url_base1)
    return df[0]

In [4]:
cpa("La Pampa","La Elenita",'')

,Provincia,Localidad,Código Postal,CPA,Código Telefónico
0,La Pampa,Caichue,6321,L6321XAA,NaN
1,La Pampa,Colonia el Porvenir,6321,L6321XAB,NaN
2,La Pampa,Costa del Salado,6321,L6321XAW,NaN
3,La Pampa,Dos Amigos,6321,L6321XAX,NaN
4,La Pampa,El Destino,6321,L6321XAY,NaN
5,La Pampa,El Mate,6321,L6321XAZ,NaN
6,La Pampa,El Odre,6321,L6321XBA,NaN
7,La Pampa,El Refugio,6321,L6321XBB,NaN
8,La Pampa,El Retiro,6321,L6321XBC,NaN
9,La Pampa,El Silencio,6321,L6321XBD,NaN
